In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("/kaggle/input/ccdata/CC GENERAL.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe().T

In [ ]:
df.columns

In [ ]:
df=df.drop(['CUST_ID'],axis=1)
df

In [ ]:
plt.figure(figsize=(10,50))
for i in range (len(df.columns)):
    plt.subplot(17,1,i+1)
    sns.distplot(df[df.columns[i]], kde_kws={"color": "b", "lw": 3, "label": "KDE","bw":0.1}, hist_kws={"color": "y"})
    plt.title(df.columns[i])
    plt.tight_layout()

In [ ]:
correlations = df.corr()
f, ax = plt.subplots(figsize = (30,15))
sns.heatmap(correlations, annot = True,cmap='YlGnBu')

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
dfc=imp_mean.fit_transform(df)
dfc

In [ ]:
null_cols = df.columns[df.isnull().any().tolist()]
null_cols

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer()
df_imp = pd.DataFrame(imputer.fit_transform(df[null_cols]), columns = null_cols)
df = df.fillna(df_imp)

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = pd.DataFrame(scale.fit_transform(df))
X

In [ ]:
wcss = []
for i in range(1,10):
    kmeans = KMeans(n_clusters = i, init='k-means++')
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:

plt.plot(range(1,10), wcss)
plt.title('Cluster Plot')
plt.xlabel('Number of Cluster')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 7, init='k-means++')
y_pred = kmeans.fit_predict(X)
np.bincount(y_pred)
labels = kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2, random_state=24)
X = pca.fit_transform(X)
X

In [ ]:
pca_df = pd.DataFrame(data = X , columns = ['pca1','pca2'])
pca_df.head()

In [ ]:

pca_df = pd.concat([pca_df, pd.DataFrame({'cluster': labels})], axis = 1)
pca_df.head()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X[:, 0], y=X[:, 1],  palette='viridis')
plt.title('Scatterplot of Data Points with Clusters')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend(title='Clusters', loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
ax = sns.scatterplot(x = "pca1", y = "pca2", hue = "cluster", data = pca_df , palette = ['green', 'red', 'blue', 'pink', 'yellow', 'purple', 'black'])
plt.show()

In [ ]:
for i in range(7):
    plt.scatter(X[y_pred==i, 0],X[y_pred==i, 1],s=50,label=f'Cluster {i}')
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], color='black',marker='*', label='Centroids')
#plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], s=300, label='Centroids')
plt.legend()
plt.show()

In [ ]:
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 
from matplotlib import pyplot as plt 
from sklearn import manifold, datasets 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.datasets import make_blobs 
%matplotlib inline

In [ ]:
X1, y1 = make_blobs()

In [ ]:
plt.scatter(X1[:, 0], X1[:, 1], marker='o') 

In [ ]:
agglom = AgglomerativeClustering(n_clusters = 7, linkage = 'average')
agglom.fit(X1,y1)

In [ ]:
dist_matrix = distance_matrix(X1,X1) 
print(dist_matrix)

In [ ]:
Z = hierarchy.linkage(dist_matrix, 'complete')
Z

In [ ]:
dendro = hierarchy.dendrogram(Z)

In [ ]:
Z=hierarchy.linkage(dist_matrix,'average')
den=hierarchy.dendrogram(Z)